<a href="https://colab.research.google.com/github/easyri/paintings/blob/master/multitask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import pandas as pd
import requests

from tensorflow import keras
from tensorflow.keras import layers
# ! pip install git+https://github.com/keras-team/autokeras.git@master
# ! pip install git+https://github.com/keras-team/keras-tuner.git@master
# ! pip install tensorflow==2.2.0
import os, glob, cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import resnet50
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from keras.layers import Flatten, Input
import keras 
from keras import layers
import torch
from torch import nn
from collections import OrderedDict
import argparse
import os
from torchvision import transforms, utils, datasets, models
from torch.utils.data import DataLoader, ConcatDataset, Dataset
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import time
import glob
import copy
import shutil
import sys
from matplotlib.ticker import MaxNLocator
from keras.applications.vgg16 import VGG16
from keras.layers import Reshape, Concatenate
from collections import defaultdict
import string
from os import listdir
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from pickle import dump
from sklearn.model_selection import train_test_split
from pickle import load, dump
import argparse
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Reshape, Concatenate

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

EMBEDDING_DIM = 256

lstm_layers = 1
dropout_rate = 0.5
learning_rate = 0.001

In [ ]:
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

# load a pre-defined list of photo identifiers
def load_set(filename):
  doc = load_doc(filename)
  dataset = list()
  # process line by line
  for line in doc.split('\n'):
    # skip empty lines
    if len(line) < 1:
      continue
    # get the image identifier
    identifier = line.split('.')[0]
    dataset.append(identifier)
  return set(dataset)



def load_photo_arrays(filename, dataset):
  # load all features
  all = load(open(filename, 'rb'))
  # filter features
  features = {k: all[k] for k in dataset}
  return features

In [ ]:
# calculate the length of the description with the most words
def max_len(descriptions):
  lines = to_lines(descriptions)
  return max(len(d.split()) for d in lines)


  # convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
  all_desc = list()
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

  
def load_clean_descriptions(filename, dataset):
  # load document
  doc = load_doc(filename)
  descriptions = dict()
  for line in doc.split('\n'):
    # split line by white space
    tokens = line.split()
    # split id from description
    image_id, image_desc = tokens[0], tokens[1:]
    # skip images not in the set
    if image_id in dataset:
      # create list
      if image_id not in descriptions:
        descriptions[image_id] = list()
      # wrap description in tokens
      desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
      # store
      descriptions[image_id].append(desc)
  return descriptions




  # convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
  all_desc = list()
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
  lines = to_lines(descriptions)
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


flikr

In [ ]:

ftrain_names = '/content/drive/My Drive/paintings/Flickr_8k.trainImages.txt'
ftest_names = '/content/drive/My Drive/paintings/Flickr_8k.devImages.txt'
ftrain = load_set(ftrain_names)
ftest = load_set(ftest_names)
ftrain_features = load_photo_arrays('/content/drive/My Drive/paintings/flikr_features.pkl', ftrain)
ftest_features = load_photo_arrays('/content/drive/My Drive/paintings/flikr_features.pkl', ftest)
print(len(ftrain_features.keys()))
print(len(ftest_features.keys()))

6000
1000


In [ ]:
ftokenizer = load(open('/content/drive/My Drive/paintings/flikr_tokenizer.pkl', 'rb'))
findex_word = load(open('/content/drive/My Drive/paintings/flikr_index_word.pkl', 'rb'))
fvocab_size = len(ftokenizer.word_index) + 1
print('Vocabulary Size: %d' % fvocab_size)

Vocabulary Size: 7579


In [ ]:
ftrain_descriptions = load_clean_descriptions('/content/drive/My Drive/paintings/flikr_descr.txt', ftrain)
ftest_descriptions = load_clean_descriptions('/content/drive/My Drive/paintings/flikr_descr.txt', ftest)
fmax_length = max_len(ftrain_descriptions)
print('Description Length: %d' % fmax_length)

Description Length: 34


wiki

In [ ]:
wfeatures = load(open('/content/drive/My Drive/paintings/wikiart/real_features.pkl', 'rb'))
wnames = list(wfeatures.keys())
wtrain, wtest = train_test_split(wnames, random_state=0)
# print(wtrain, wtest, sep=' ')
wtrain_features = {k: wfeatures[k] for k in wtrain}
wtest_features = {k: wfeatures[k] for k in wtest}
wtrain_descriptions = load_clean_descriptions('/content/drive/MyDrive/paintings/real_descr.txt', wtrain)
# print(train_descriptions)
wtest_descriptions = load_clean_descriptions('/content/drive/MyDrive/paintings/real_descr.txt', wtest)
wmax_length = max_len(wtrain_descriptions)
# print(max_length)
wtokenizer = create_tokenizer(wtrain_descriptions)
# dump(tokenizer, open('/content/drive/My Drive/paintings/real_tokenizer.pkl', 'wb'))
windex_word = {v: k for k, v in wtokenizer.word_index.items()}
# dump(index_word, open('/content/drive/My Drive/paintings/real_index_word.pkl', 'wb'))

wvocab_size = len(wtokenizer.word_index) + 1
print('Vocabulary Size: %d' % wvocab_size)

In [ ]:
new_wtrain, new_wtest = wtrain[:6000], wtest[:1000]
wtrain_features = {k: wfeatures[k] for k in new_wtrain}
wtest_features = {k: wfeatures[k] for k in new_wtest}
wtrain_descriptions = load_clean_descriptions('/content/drive/MyDrive/paintings/real_descr.txt', new_wtrain)
# print(train_descriptions)
wtest_descriptions = load_clean_descriptions('/content/drive/MyDrive/paintings/real_descr.txt', new_wtest)
wmax_length = max_len(wtrain_descriptions)
# print(max_length)
wtokenizer = create_tokenizer(wtrain_descriptions)
# dump(tokenizer, open('/content/drive/My Drive/paintings/real_tokenizer.pkl', 'wb'))
windex_word = {v: k for k, v in wtokenizer.word_index.items()}
# dump(index_word, open('/content/drive/My Drive/paintings/real_index_word.pkl', 'wb'))

wvocab_size = len(wtokenizer.word_index) + 1
print('Vocabulary Size: %d' % wvocab_size)

Vocabulary Size: 5582


In [ ]:
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo):
  vocab_size = len(tokenizer.word_index) + 1
  X1, X2, y = [], [], []
  # walk through each description for the image
  for desc in desc_list:
    # encode the sequence
    seq = tokenizer.texts_to_sequences([desc])[0]
    # split one sequence into multiple X,y pairs
    for i in range(1, len(seq)):
      # split into input and output pair
      in_seq, out_seq = seq[:i], seq[i]
      # pad input sequence
      in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
      # encode output sequence
      out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
      # store
      X1.append(photo)
      X2.append(in_seq)
      y.append(out_seq)
  return np.array(X1), np.array(X2), np.array(y)

def data_generator(descriptions, photos, tokenizer, max_length, n_step = 1):
  # loop for ever over images
  while True:
    # loop over photo identifiers in the dataset
    keys = list(descriptions.keys())
    for i in range(0, len(keys), n_step):
      Ximages, XSeq, y = list(), list(),list()
      for j in range(i, min(len(keys), i+n_step)):
        image_id = keys[j]
        # retrieve the photo feature
        photo = photos[image_id][0]
        desc_list = descriptions[image_id]
        # print(desc_list)
        in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo)
        # print(in_img, in_seq, out_word)
        for k in range(len(in_img)):
          Ximages.append(in_img[k])
          XSeq.append(in_seq[k])
          y.append(out_word[k])
        
      yield ([np.array(Ximages), np.array(XSeq)], np.array(y))


def double_generator(descriptions1, photos1, tokenizer1, max_length1,
                       descriptions2, photos2, tokenizer2, max_length2, n_step=1):
  while True:
    # loop over photo identifiers in the dataset
    keys1 = list(descriptions1.keys())
    keys2 = list(descriptions2.keys())    # len(keys1) == len(keys2)
    for i in range(0, len(keys1), n_step):
      Ximages1, XSeq1, y1 = list(), list(),list()
      Ximages2, XSeq2, y2 = list(), list(),list()
      for j in range(i, min(len(keys1), i+n_step)):
        image_id1 = keys1[j]
        # retrieve the photo feature
        photo1 = photos1[image_id1][0]
        desc_list1 = descriptions1[image_id1]
        # print(desc_list)
        in_img1, in_seq1, out_word1 = create_sequences(tokenizer1, max_length1, desc_list1, photo1)
        # print(in_img, in_seq, out_word)
        for k in range(len(in_img1)):
          Ximages1.append(in_img1[k])
          XSeq1.append(in_seq1[k])
          y1.append(out_word1[k])
        # print('Ximages1', Ximages1)
        # print('Xseq1', XSeq1)
        # print('y1', y1)
      for j in range(i, min(len(keys2), i+n_step)):
        image_id2 = keys2[j]
        # retrieve the photo feature
        photo2 = photos2[image_id2][0]
        desc_list2 = descriptions2[image_id2]
        # print(desc_list)
        in_img2, in_seq2, out_word2 = create_sequences(tokenizer2, max_length2, desc_list2, photo2)
        # print(in_img, in_seq, out_word)
        for k in range(len(in_img2)):
          Ximages2.append(in_img2[k])
          XSeq2.append(in_seq2[k])
          y2.append(out_word2[k])
        # print('Ximages2', Ximages2)
        # print('Xseq2', XSeq2)
        # print('y2', y2)
      yield ([np.array(Ximages1), np.array(XSeq1), np.array(Ximages2), np.array(XSeq2)], [np.array(y1), np.array(y2)])
    



# define the captioning model
def define_model(vocab_size1, max_length1, vocab_size2, max_length2):

  # flikr
  inputs1 = Input(shape=(4096,))
  print(inputs1.shape)
  fe1_1 = Dropout(0.5)(inputs1)
  fe2_1 = Dense(EMBEDDING_DIM, activation='relu')(fe1_1)
  fe3_1 = RepeatVector(max_length1)(fe2_1)

  inputs2 = Input(shape=(max_length1,))
  print(inputs2.shape)
  emb2_1 = Embedding(vocab_size1, EMBEDDING_DIM, mask_zero=True)(inputs2)
  
  merged1 = concatenate([fe3_1, emb2_1], name='concat1')
  lm2_1 = LSTM(500, return_sequences=False)(merged1)
  # print(lm2_1.shape)
  # lm2_1 = Flatten()(lm2_1)


  # wiki
  inputs3 = Input(shape=(4096,))
  fe1_2 = Dropout(0.5)(inputs3)
  fe2_2 = Dense(EMBEDDING_DIM, activation='relu')(fe1_2)
  fe3_2 = RepeatVector(max_length2)(fe2_2)
  
  inputs4 = Input(shape=(max_length2,))
  emb2_2 = Embedding(vocab_size2, EMBEDDING_DIM, mask_zero=True)(inputs4)
  
  merged2 = concatenate([fe3_2, emb2_2], name='concat2')
  lm2_2 = LSTM(500, return_sequences=False)(merged2)
  # print(lm2_2.shape)
  # lm2_2 = Flatten()(lm2_2)

# merge
  merged3 = concatenate([lm2_1, lm2_2], axis=0, name='concat3')
  outputs = Dense(vocab_size1, activation='softmax')(merged3)
  outputs1 = Dense(vocab_size2, activation='softmax')(merged3)

  # tie it together [image, seq] [word]
  model = Model(inputs=[inputs1, inputs2, inputs3, inputs4], outputs=[outputs, outputs1])
  model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
  print(model.summary())
  # plot_model(model, show_shapes=True, to_file='model.png')
  return model

In [ ]:

model = define_model(fvocab_size, fmax_length, wvocab_size, wmax_length)

(None, 4096)
(None, 34)
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 4096)]       0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 4096)]       0                                            
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 4096)         0           input_21[0][0]                   
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 4096)         0           input_23[0][0]                   
____________________________________________________________________

In [ ]:
train_generator1 = data_generator(ftrain_descriptions, ftrain_features, ftokenizer, fmax_length)
val_generator1 = data_generator(ftest_descriptions, ftest_features, ftokenizer, fmax_length)

train_generator2 = data_generator(wtrain_descriptions, wtrain_features, wtokenizer, wmax_length)
val_generator2 = data_generator(wtest_descriptions, wtest_features, wtokenizer, wmax_length)

In [ ]:
train_generator = double_generator(ftrain_descriptions, ftrain_features, ftokenizer, fmax_length,
                                   wtrain_descriptions, wtrain_features, wtokenizer, wmax_length)
val_generator = double_generator(ftest_descriptions, ftest_features, ftokenizer, fmax_length,
                                 wtest_descriptions, wtest_features, wtokenizer, wmax_length)

In [ ]:
  # define checkpoint callback
filepath = '/content/drive/My Drive/paintings/multi/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,save_freq='epoch',
              save_best_only=False, save_weights_only=False, mode='min')

steps = len(ftrain_descriptions)
val_steps = len(ftest_descriptions)
  # create the data generator


  # fit model
 
model.fit(train_generator, epochs=20,  verbose=1, steps_per_epoch=steps, validation_steps=val_steps,
      callbacks=[checkpoint], validation_data=val_generator)

try:
    model.save('/content/drive/My Drive/paintings/multi/multi_wholeModel.h5', overwrite=True)
    model.save_weights('/content/drive/My Drive/paintings/multi/multi_weights.h5',overwrite=True)
except:
    print("Error in saving model.")
print("Training complete...\n")

Epoch 1/20


InvalidArgumentError: ignored